# Training Models

# A. Linear Regression:

* Generally, a linear model makes a prediction by simply computing a weighted sum of the input features, plus a constant called the **bias** term (also called the **intercept** term):

    * $\hat{y}\ =\ \theta_0\ +\ \theta_1x_1\ +\ ...\ +\ \theta_nx_n$ where:
        * $\hat{y}$ = predicted value.
        * $n$ = number of features.
        * $x_i$ = $i^{th}$ feature value.
        * $\theta_j$ = $j^{th}$ model parameter, including the bias term $\theta_0$ and the feature weights $\theta_1,\theta_2,...,\theta_n$.

    * In vectorized form: $\hat{y}\ =\ h_\theta(x)\ =\ \theta.x$, where:
        * $h_\theta$ = hypothesis function, using the model parameter $\theta$.
        * $\theta$ = model's parameter vector, containing the bias term $\theta_0$ and feature weights $\theta_1$ to $\theta_n$.
        * $x$ is the instance's feature vector, containing $x_0$ to $x_n$, with $x_0$ always equal to 1.
        * $\theta.x$ = dot product of the vectors $\theta$ and $x$, which is equal to $\theta_0x_0 +\theta_1x_1 + \theta_2x_2 +...+ \theta_nx_n$